In [1]:
import parselmouth
import parselmouth.praat as praat
import numpy as np
import os

In [ ]:
folder_path = r'C:\Users\ivyap\Desktop\25SU\TEAMLAB\LA\ASVspoof2019_LA_train\flac'
output_pitch = r'C:\Users\ivyap\Desktop\25SU\TEAMLAB\toy_feature\feature_pitch'
output_hnr = r'C:\Users\ivyap\Desktop\25SU\TEAMLAB\toy_feature\feature_hnr'
output_jitter = r'C:\Users\ivyap\Desktop\25SU\TEAMLAB\toy_feature\feature_jitter'
output_shimmer = r'C:\Users\ivyap\Desktop\25SU\TEAMLAB\toy_feature\feature_shimmer'

if output_pitch and not os.path.exists(output_pitch):
    os.makedirs(output_pitch)
if output_pitch and not os.path.exists(output_hnr):
    os.makedirs(output_hnr)
if output_pitch and not os.path.exists(output_jitter):
    os.makedirs(output_jitter)
if output_pitch and not os.path.exists(output_shimmer):
    os.makedirs(output_shimmer)

In [ ]:

n_file = 0
for filename in os.listdir(folder_path):
    full_file_path = os.path.join(folder_path, filename)
    
    # output path
    base_filename = os.path.basename(filename).replace('.flac', '')
    pitch_output_path = os.path.join(output_pitch, f'{base_filename}_pitch.npy')
    hnr_output_path = os.path.join(output_hnr, f'{base_filename}_hnr.npy')
    jitter_output_path = os.path.join(output_jitter, f'{base_filename}_jitter.npy')
    shimmer_output_path = os.path.join(output_shimmer, f'{base_filename}_shimmer.npy')

    try:
        sound = parselmouth.Sound(full_file_path)

        # get the pitch
        pitch = sound.to_pitch(
            time_step=0.01,
            pitch_floor=75,
            pitch_ceiling=600,
        )
        pitch_values = pitch.selected_array['frequency']
        pitch_values[pitch_values == 0] = np.nan

        np.save(pitch_output_path, pitch_values)
        print(f"Processed and saved {filename}")


        # get the hnr
        harmonicity = sound.to_harmonicity(time_step=0.01)
        hnr_values = harmonicity.values.T


        # get jitter and shimmer
        pitch_floor = 75.0  # Minimum pitch in Hz
        pitch_ceiling = 600.0 # Maximum pitch in Hz
        total_duration = sound.get_duration()
        voice_report = praat.call(sound, "To Voice", 0.0, total_duration, pitch_floor, pitch_ceiling, 0.3, 4.5, 0.01, 0.1, 0.01, 0.3, 1.6, 0.06, 0.16, 0.1, 0.3, 0.1, 0.3)
        local_jitter = praat.call(voice_report, "Get jitter (local)...", 0.0, 0.0, 0.07, 0.1, 1.6) * 100 # Get as percentage
        local_shimmer = praat.call(voice_report, "Get shimmer (local)...", 0.0, 0.0, 0.7, 0.1, 1.6) # Get as percentage


    except Exception as e:
        print(f"Skipping {filename} due to error: {e}")

    n_file += 1
    if n_file == 10: break

Processed and saved LA_T_1000137.flac
Processed and saved LA_T_1000406.flac
Processed and saved LA_T_1000648.flac
Processed and saved LA_T_1000824.flac
Processed and saved LA_T_1001074.flac
Processed and saved LA_T_1001114.flac
Processed and saved LA_T_1001169.flac
Processed and saved LA_T_1001718.flac
Processed and saved LA_T_1001871.flac
Processed and saved LA_T_1002656.flac


In [12]:
for filename in os.listdir(output_dir):
    full_file_path = os.path.join(output_dir, filename)
    try:
        loaded_features = np.load(full_file_path)

        print(f"Successfully loaded data from: {full_file_path}")

        # --- Inspect the loaded data ---

        # 1. Check the type (it should be a NumPy array)
        print(f"Data type of the loaded object: {type(loaded_features)}")

        # 2. Check the shape (dimensions) of the array
        # For pitch, this will likely be (number_of_frames,)
        # For MFCCs, this might be (number_of_frames, number_of_mfcc_coefficients)
        print(f"Shape of the loaded array: {loaded_features.shape}")

        # 3. Check the data type of the elements in the array
        print(f"Data type of elements: {loaded_features.dtype}")

        # 4. Check the number of dimensions
        print(f"Number of dimensions: {loaded_features.ndim}")

        # 5. Print the first few values (if it's a 1D array like pitch)
        if loaded_features.ndim == 1:
            print(f"First 10 values: {loaded_features[:10]}")
            print(f"Last 10 values: {loaded_features[-10:]}")
        # Or print a slice if it's multi-dimensional like MFCCs
        elif loaded_features.ndim == 2:
            print(f"First 5 rows (frames) and first 5 columns (coefficients):\n{loaded_features[:5, :5]}")

        # 6. Get basic statistics (ignoring NaN values if you replaced zeros with NaN)
        print(f"Minimum value: {np.nanmin(loaded_features)}") # Use nanmin to ignore NaNs
        print(f"Maximum value: {np.nanmax(loaded_features)}") # Use nanmax to ignore NaNs
        print(f"Mean value: {np.nanmean(loaded_features)}")   # Use nanmean to ignore NaNs
        print(f"Standard deviation: {np.nanstd(loaded_features)}") # Use nanstd to ignore NaNs
        print(f"Number of non-NaN values: {np.count_nonzero(~np.isnan(loaded_features))}") # Count non-NaNs
        print(f"Number of NaN values: {np.count_nonzero(np.isnan(loaded_features))}")     # Count NaNs


    except FileNotFoundError:
        print(f"Error: The file was not found at {full_file_path}")
    except Exception as e:
        print(f"An error occurred while loading/inspecting the file: {e}")



Successfully loaded data from: C:\Users\ivyap\Desktop\25SU\TEAMLAB\toy_feature\feature_pitch\LA_T_1000137_pitch.npy
Data type of the loaded object: <class 'numpy.ndarray'>
Shape of the loaded array: (247,)
Data type of elements: float64
Number of dimensions: 1
First 10 values: [nan nan nan nan nan nan nan nan nan nan]
Last 10 values: [nan nan nan nan nan nan nan nan nan nan]
Minimum value: 94.62531674777355
Maximum value: 181.83479404934673
Mean value: 129.70324577173764
Standard deviation: 19.223900031522092
Number of non-NaN values: 106
Number of NaN values: 141
Successfully loaded data from: C:\Users\ivyap\Desktop\25SU\TEAMLAB\toy_feature\feature_pitch\LA_T_1000406_pitch.npy
Data type of the loaded object: <class 'numpy.ndarray'>
Shape of the loaded array: (282,)
Data type of elements: float64
Number of dimensions: 1
First 10 values: [nan nan nan nan nan nan nan nan nan nan]
Last 10 values: [nan nan nan nan nan nan nan nan nan nan]
Minimum value: 109.62684748606927
Maximum value: 18